In [1]:
import os

from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

graph = Neo4jGraph()

In [2]:
graph.query('MATCH (n) DETACH DELETE n;')
# graph.query('drop index chunk_index;')

[]

In [3]:
from langchain_community.document_loaders import PyPDFLoader
pages = []
dir = '../data'
for filename in os.listdir(dir):
    docs = []
    if filename.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(dir, filename))
        pages += loader.load()
    break # too much time, only load one file
pages
pages = [pages[0]]

In [4]:
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
# from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [5]:
import sys
sys.path.append("..")
from tools.graph_builder import TwlfGraphBuilder

graph_builder = TwlfGraphBuilder(graph)
graph_builder.graph_build(pages)
graph_builder.chunk_list

[{'chunk_id': '6cf3546a-e1f7-4349-9e4b-9e99810af1f0',
  'chunk_doc': Document(metadata={'source': '../data/台灣人壽美鑫美利美元利率變動型終身壽險.pdf', 'page_number': 1}, page_content='第 1 頁，共  4 頁 台灣人壽 美鑫美利美元利率變動型終身壽險   保險商品內容說明   【承保範圍】   【增值回饋分享金的給付及通知】   第十三條   本公司於本契約有效期間內之每一保單年度屆滿後，除被保險人保險年齡達 16歲前者，按第三 項約定辦理外，將依要保人於投保時所選擇下列方式之一給付增值回饋分享金：   一、購買增額繳清保險金額：選擇購買增額繳清保險金額者，以增值回饋分享金為躉繳純保險費 ，計算自該保單週年日當日起生效之增額繳清保險金額，但被保險人為受監護宣告尚未撤銷者， 應依保單條款 第十六條約定辦理。   二、現金給付：選擇現金給付者')},
 {'chunk_id': 'ac6f33a1-7699-45ad-8da1-3990f779fb56',
  'chunk_doc': Document(metadata={'source': '../data/台灣人壽美鑫美利美元利率變動型終身壽險.pdf', 'page_number': 1}, page_content='，依本契約約定 以現金給付增值回饋分享金予要保人，惟須於第 六保單年度屆滿後之每一保單週年日起，始得依本款方式給付。   三、儲存生息：選擇儲存生息者，各年度之增值回饋分享金將按各保單週年日當月之宣告利率依 據年複利方式，累積至要保人請求時給付，或至被保險人身故或本契約終止時，由本公司主動一 併給付。但在本公司給付受益人保險金而終止契約的情形，要保人未請求之增值回饋分享金及其 孳息，由該保險金受益人受領。惟須於第六保單年度屆滿後之每一保單週年日起，始得依本款方 式給付。   要保人若未選擇者，則視為選擇購買增額繳清保險金額，並得於本契約有效 期間內，以書面通知 本公司變更前項給付方式')},
 {'chunk_id': 'b4204ff1-5702-49cf-a903-97861064d3f6',
  'chunk_doc': Document(met

In [6]:
graph_builder._get_combined_chunks(graph_builder.chunk_list)

[Document(metadata={'combined_chunk_ids': ['6cf3546a-e1f7-4349-9e4b-9e99810af1f0']}, page_content='第 1 頁，共  4 頁 台灣人壽 美鑫美利美元利率變動型終身壽險   保險商品內容說明   【承保範圍】   【增值回饋分享金的給付及通知】   第十三條   本公司於本契約有效期間內之每一保單年度屆滿後，除被保險人保險年齡達 16歲前者，按第三 項約定辦理外，將依要保人於投保時所選擇下列方式之一給付增值回饋分享金：   一、購買增額繳清保險金額：選擇購買增額繳清保險金額者，以增值回饋分享金為躉繳純保險費 ，計算自該保單週年日當日起生效之增額繳清保險金額，但被保險人為受監護宣告尚未撤銷者， 應依保單條款 第十六條約定辦理。   二、現金給付：選擇現金給付者'),
 Document(metadata={'combined_chunk_ids': ['ac6f33a1-7699-45ad-8da1-3990f779fb56']}, page_content='，依本契約約定 以現金給付增值回饋分享金予要保人，惟須於第 六保單年度屆滿後之每一保單週年日起，始得依本款方式給付。   三、儲存生息：選擇儲存生息者，各年度之增值回饋分享金將按各保單週年日當月之宣告利率依 據年複利方式，累積至要保人請求時給付，或至被保險人身故或本契約終止時，由本公司主動一 併給付。但在本公司給付受益人保險金而終止契約的情形，要保人未請求之增值回饋分享金及其 孳息，由該保險金受益人受領。惟須於第六保單年度屆滿後之每一保單週年日起，始得依本款方 式給付。   要保人若未選擇者，則視為選擇購買增額繳清保險金額，並得於本契約有效 期間內，以書面通知 本公司變更前項給付方式'),
 Document(metadata={'combined_chunk_ids': ['b4204ff1-5702-49cf-a903-97861064d3f6']}, page_content='，以書面通知 本公司變更前項給付方式，惟第六保單年度屆滿前，增值回饋分享金限以購買增額繳清保險金額 辦理。  被保險人保險年齡到達 16歲前，其增值回饋分享金於繳費期間應採抵繳保險費之方式辦理。但 因繳費期間已屆滿而無法抵繳保險費者，本

In [7]:

graph_documents = graph_builder.get_graph_from_llm(llm, graph_builder.chunk_list, allowedNodes=[], allowedRelationship=[])
graph_documents

[GraphDocument(nodes=[Node(id='增值回饋分享金', type='Concept'), Node(id='要保人', type='Person'), Node(id='身故保險金', type='Concept'), Node(id='喪葬費用保險金', type='Concept'), Node(id='被保險人', type='Person'), Node(id='本公司', type='Organization'), Node(id='當年度保險金額', type='Concept'), Node(id='本契約', type='Concept'), Node(id='未成年人', type='Person'), Node(id='民國九十九年二月三日', type='Date')], relationships=[Relationship(source=Node(id='增值回饋分享金', type='Concept'), target=Node(id='要保人', type='Person'), type='通知'), Relationship(source=Node(id='本公司', type='Organization'), target=Node(id='被保險人', type='Person'), type='給付'), Relationship(source=Node(id='身故保險金', type='Concept'), target=Node(id='喪葬費用保險金', type='Concept'), type='變更'), Relationship(source=Node(id='未成年人', type='Person'), target=Node(id='被保險人', type='Person'), type='身故'), Relationship(source=Node(id='本契約', type='Concept'), target=Node(id='未成年人', type='Person'), type='訂立'), Relationship(source=Node(id='被保險人', type='Person'), target=Node(id='民國九十九年二月三日', type='Date

In [8]:
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

In [9]:
chunks_and_graphDocuments_list = graph_builder.get_chunk_and_graphDocument(graph_documents)
chunks_and_graphDocuments_list[0]

{'graph_doc': GraphDocument(nodes=[Node(id='增值回饋分享金', type='Concept'), Node(id='要保人', type='Person'), Node(id='身故保險金', type='Concept'), Node(id='喪葬費用保險金', type='Concept'), Node(id='被保險人', type='Person'), Node(id='本公司', type='Organization'), Node(id='當年度保險金額', type='Concept'), Node(id='本契約', type='Concept'), Node(id='未成年人', type='Person'), Node(id='民國九十九年二月三日', type='Date')], relationships=[Relationship(source=Node(id='增值回饋分享金', type='Concept'), target=Node(id='要保人', type='Person'), type='通知'), Relationship(source=Node(id='本公司', type='Organization'), target=Node(id='被保險人', type='Person'), type='給付'), Relationship(source=Node(id='身故保險金', type='Concept'), target=Node(id='喪葬費用保險金', type='Concept'), type='變更'), Relationship(source=Node(id='未成年人', type='Person'), target=Node(id='被保險人', type='Person'), type='身故'), Relationship(source=Node(id='本契約', type='Concept'), target=Node(id='未成年人', type='Person'), type='訂立'), Relationship(source=Node(id='被保險人', type='Person'), target=Node(id='民國九十九年二月三日

In [10]:
graph_builder.merge_relationship_between_chunk_and_entites(chunks_and_graphDocuments_list)

In [11]:
from langchain_community.vectorstores import Neo4jVector

from langchain_openai import AzureOpenAIEmbeddings

embedding = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
    azure_deployment='text-embedding-ada-002',
    openai_api_version='2023-05-15'
)

vector = Neo4jVector.from_existing_graph(
    embedding,
    node_label='Chunk',
    text_node_properties=['content'],
    embedding_node_property='embedding'
)